In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

##### 128x2 Variable Threshold Sequence Dataset

In [ ]:
thresholds = [
    [25, 26, 27, 28, 28],
    [21, 17, 21, 20, 21],
    [19, 14, 22, 25, 25],
    [25, 22, 27, 24, 27],
    [24, 22, 24, 26, 25]
]

counts = [
    [156, 145, 154, 160, 142],
    [149, 130, 156, 136, 120],
    [142, 103, 142, 152, 137],
    [145, 144, 159, 149, 142],
    [137, 138, 136, 124, 134]
]

locations = ["A", "B", "C", "D", "E"]

In [ ]:
labels = ["User 1", "User 2", "User 3", "User 4", "User 5"]
colors = plt.get_cmap('Set3').colors

counts = np.array(counts)
x = np.arange(5)

# counts: stacked barplot
fig, ax = plt.subplots()
bottom = np.zeros(5)
for i, row in enumerate(counts):
    ax.bar(x, row, bottom=bottom, label=labels[i], color = colors[i])
    bottom += row 

ax.set_xticks(x)
ax.set_xticklabels(locations)
ax.set_xlabel("Location")
ax.set_ylabel("Number of Sequences")
ax.legend(loc="upper center", bbox_to_anchor=(0.5, 1.12), ncol=5)
plt.show()

# thresholds: line plot

for i in range(5):
    plt.plot(locations, thresholds[i], label=labels[i], color = colors[i])
plt.legend()
plt.xlabel("Location")
plt.ylabel("Peak Acceleration Threshold")
plt.show()

##### Single sequence (normalized)

In [ ]:
def sequence_norm(user=1, location="A", instance=1):
    A1 = pd.read_csv(f"/mnt/c/Users/olivi/OneDrive - Amherst College/6 Summer 2025/surface-detection/sequences/varied-threshold/User{user}_Location{location}_Normal_{instance}.dat", header=None)
    plt.plot(list(range(1, 129)), A1[0] / A1[0].max(), label='acceleration')
    plt.plot(list(range(1, 129)), A1[1] / A1[1].max(), color='red', label="Gyroscope")
    plt.title("Normalized Accelerometer, Gyroscope for Sequence")
    plt.legend()
    plt.show()

sequence_norm()

##### Confusion heatmap

In [ ]:
nn = "ver3"
train = pd.read_csv(f"/mnt/c/Users/olivi/OneDrive - Amherst College/6 Summer 2025/surface-detection/logs/{nn}_accuracy_train.txt")
val = pd.read_csv(f"/mnt/c/Users/olivi/OneDrive - Amherst College/6 Summer 2025/surface-detection/logs/{nn}_accuracy_val.txt")
test = pd.read_csv(f"/mnt/c/Users/olivi/OneDrive - Amherst College/6 Summer 2025/surface-detection/logs/{nn}_accuracy_test.txt")

In [ ]:
# confusion matrix heatmap, normalized over true (rows)
def confusion_heatmap(set=train, netname="Ver3", setname="Training"):
    sklearn.metrics.ConfusionMatrixDisplay.from_predictions(set["True"], set["Predicted"], 
                                                            cmap='cividis', 
                                                            display_labels=["A", "B", "C", "D", "E"],
                                                            normalize="true")
    plt.xlabel("Predicted location")
    plt.ylabel("True location")
    plt.title(f"{netname} Predictions on {setname} set")
    plt.show()

In [ ]:
confusion_heatmap(train, "Ver3", "Training")
confusion_heatmap(val, "Ver3", "Validation")
confusion_heatmap(test, "Ver3", "Test")